<a href="https://colab.research.google.com/github/lejejefr/ECM_2526_FinalProject/blob/main/Scripts/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ce script sert à entrainer notre modèle et à l'utiliser s'il est déjà entrainé


In [2]:
#ce bloc sert à installer les packages python dont nous avons besoin
!pip install pandas numpy scikit-learn matplotlib

In [3]:
#dans ce bloc, nous installons les différentes bibliothèques dont nous aurons besoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [4]:
# cette fonction calcule l'aire d'une zone rectangulaire en fonction des coordonnées des bords
#lat1, lat2, lon1 et lon2 sont respectivement les latitudes et longitudes des côtés de la zone observée.
#la fonction renvoit une aire en km2

def area(lat1, lat2, lon1, lon2, R = 6378):
    lat1 = np.pi * lat1 / 180.0
    lat2 = np.pi * lat2 / 180.0
    lon1 = np.pi * lon1 / 180.0
    lon2 = np.pi * lon2 / 180.0

    h = max(np.sin(lat1), np.sin(lat2)) - min(np.sin(lat1), np.sin(lat2))
    r = (max(lon1, lon2) - min(lon1,lon2)) / (2 * np.pi)

    return 2 * np.pi * (R ** 2) * h * r

print(area(41.644818,42.022548,-87.904123,-87.525759))
#-->le rectangle couvert par les données est de 1319.5307651175892km2, soit environ 2 fois l'air de chicago. Si on regarde une carte de la ville, on voit que le plus petit
#rectangle contenant la ville peut la contenir à peu près 2 fois, donc a priori la fonction calculant l'aire en fonction des coordonnées fonctionne bien.

1319.5307651175892


In [26]:
#ce bloc permet d'importer la base de donnée propre
url = "https://raw.githubusercontent.com/lejejefr/ECM_2526_FinalProject/refs/heads/main/Data/CleanData.csv"
def get_data_frame_from_url(required_columns = [], _url : str = url):
  df = pd.read_csv(url)
  df = df[required_columns]
  df = df.dropna()
  df.describe()
  return df
#la ligne suivante sert uniquement à vérifier que la base de données a bien été importée
#-->

Index(['ID', 'time', 'day of week', 'day of year', 'latitude', 'longitude'], dtype='object')
Index(['ID', 'time', 'day of week', 'day of year', 'latitude', 'longitude'], dtype='object')


Dans cette partie, on a essayé de mettre en place une random forest pour évaluer le nombre de meutres au voisinage du point étudié


In [6]:
#ce bloc permet de générer des données labélisées utilisables pour l'entraînement et le test
min_lat = 41.644818
max_lat = 42.022548

min_long = -87.904123
max_long = -87.525759

min_time = 0.0
max_time = 86340.0

space_division = 50
time_division = 24

latitude_span = (max_lat - min_lat) / space_division
longitude_span = (max_long - min_long) / space_division
time_span = (max_time - min_time) / time_division

def calculate_density(row):
  lat_center = row["latitude"]
  lon_center = row["longitude"]
  time_center = row["time"]
  dow = row["day of week"]
  doy = row["day of year"]

  lat_min_zone = max(lat_center - latitude_span/2, min_lat)
  lat_max_zone = min(lat_center + latitude_span/2, max_lat)

  lon_min_zone = max(lon_center - longitude_span/2, min_long)
  lon_max_zone = min(lon_center + longitude_span/2, max_long)

  time_min_zone = max(time_center - time_span/2, min_time)
  time_max_zone = min(time_center + time_span/2, max_time)
  subset = df[
      (df["latitude"].between(lat_min_zone, lat_max_zone)) &
      (df["longitude"].between(lon_min_zone, lon_max_zone)) &
      (df["time"].between(time_min_zone, time_max_zone))
      #(df["day of week"] == dow) &
      #(df["day of year"] == doy)
      ]
  count = len(subset)
  volume = ((time_max_zone - time_min_zone) / 3600) * area(lat_min_zone,lat_max_zone,lon_min_zone,lon_max_zone)
  if volume <= 0.00001:
    return 0
  return count




labeled_df = pd.DataFrame({
    "latitude": np.random.uniform(min_lat, max_lat, 15000),
    "longitude": np.random.uniform(min_long, max_long, 15000),
    "time": np.random.uniform(0.000000, 86340.000000, 15000),
    "day of week": np.random.randint(0, 6, 15000),
    "day of year": np.random.randint(1, 366, 15000),
})
labeled_df["density"] = labeled_df.apply(calculate_density, axis = 1)

labeled_df.describe()

NameError: name 'df' is not defined

In [ ]:
labeled_df.describe()

In [ ]:
# Trier les données
sorted_density = np.sort(labeled_df["density"])

# Calculer les quantiles (0 à 1)
quantiles = np.arange(1, len(sorted_density)+1) / len(sorted_density)

# Plot
plt.figure(figsize=(6,4))
plt.plot(quantiles, sorted_density, marker='.', linestyle='none')
plt.xlabel("Quantile")
plt.ylabel("Density")
plt.title("Quantile plot of density")
plt.grid(True)
plt.show()

In [ ]:
#lat_bin = pd.cut(labeled_df["latitude"], bins=10, labels=False)
#lon_bin = pd.cut(labeled_df["longitude"], bins=10, labels=False)
#time_bin = pd.cut(labeled_df["time"], bins=6, labels=False)

density_bin = pd.cut(labeled_df["density"], bins=3, labels=False)

#strat = lat_bin.astype(str) + "_" + lon_bin.astype(str) + "_" + time_bin.astype(str) + "_" + density_bin.astype(str)

X = labeled_df.drop('density',axis = 1)
Y = labeled_df['density']

print(len(X))
print(len(Y))
print(len(density_bin))

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=None
)
print(X_train.describe())
print(X_test.describe())
print(Y_train.describe())
print(Y_test.describe())

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train,Y_train)

In [ ]:
Y_pred = rf.predict(X_test)
print(Y_pred)

In [ ]:
prediction_dataframe = pd.DataFrame({
    'prediction': Y_pred
})

error_df = pd.DataFrame()
print(Y_test.describe())
error_df["error"] = (Y_pred - Y_test)**2
print(error_df.describe())

In [ ]:
# Trier les données
sorted_error = np.sort(error_df["error"])

# Calculer les quantiles (0 à 1)
quantiles = np.arange(1, len(sorted_error)+1) / len(sorted_error)

# Plot
plt.figure(figsize=(6,4))
plt.plot(quantiles, sorted_error, marker='.', linestyle='none')
plt.xlabel("Quantile")
plt.ylabel("Squared Error")
plt.title("Quantile plot of error")
plt.grid(True)
plt.show()

In [ ]:
accuracy = mean_squared_error(Y_test,Y_pred)
print(accuracy)

Dans cette partie, nous avons mis en place différents modèles KDE pour évaluer la densité de probabilité qui correspond à notre distribution

In [7]:
#kernels possibles : {‘gaussian’, ‘tophat’, ‘epanechnikov’, ‘exponential’, ‘linear’, ‘cosine’}
def get_train_test_data_frames(X, _test_size : float = 0.2):
  return train_test_split(X, test_size=_test_size, random_state=1)

def get_trained_KDE(train_data, _kernel : str = 'gaussian',):
  model = KernelDensity(kernel=_kernel).fit(train_data)
  return model

In [8]:
def train_estimator(estimator, data):
  trained_estimator = estimator.fit(data)
  return trained_estimator

def test_estimator(estimator, data, _test_size : float = 0.2):
  train_data, test_data = train_test_split(data, test_size = _test_size)
  trained_estimator = train_estimator(estimator,train_data)
  score = trained_estimator.score(test_data)
  print(score)

test_estimator(KernelDensity(),get_data_frame_from_url())
#--> score de likelyhood pas fou

-76626338915.95229
